In [3]:
from collections import Counter
import math
import csv

#########################################################################################
##########################         Parametres         ###################################
#########################################################################################
#"Number of result"
k=0
#"Absolute magnitude parameter"
h = 0
#"object diameter (from equivalent sphere) km Unit"
diameter = 0
#"Geometric albedo"
albedo = 0
#"1-sigma uncertainty in object diameter km Unit"
diameter_sigma = 0
#"Eccentricity"
e = 0
#"Semi-major axis au Unit"
a = 0
#"perihelion distance au Unit"
q = 0
#"inclination; angle with respect to x-y ecliptic plane"
i = 0
#"Minimum orbit intersection distance"
moid = 0
#########################################################################################
#########################################################################################
#########################################################################################

#On mes les parametre dans un tableaux query pour pouvoir ensuite le reutiliser
query = [h, diameter, albedo, diameter_sigma, e, a, q, i, moid]

#Fonction qui permet de classer les données en fonction de leur similarité avec d'autres données connues
#data : un ensemble de données connues pour effectuer la classification.
#query : un point de données à classer.
#k : le nombre de voisins les plus proches à considérer lors de la classification.
#distance_fn : une fonction qui calcule la distance entre deux points de données.
#choice_fn : une fonction qui calcule la classe majoritaire des k plus proches voisins.
def knn(data, query, k, distance_fn, choice_fn):
    neighbor_distances_and_indices = []
    for index , example in enumerate(data):
        distance = distance_fn(example[:-1], query)
        neighbor_distances_and_indices.append((distance, index))
    sorted_neighbor_distances_and_indices = sorted(neighbor_distances_and_indices)
    k_nearest_distances_and_indices = sorted_neighbor_distances_and_indices[:k]
    k_nearest_labels = [data[i][-1] for distance, i in k_nearest_distances_and_indices]
    return k_nearest_distances_and_indices , choice_fn(k_nearest_labels) 

#Fonction qui permet de calculer la distance euclidienne entre deux points
#point1 : un point de données.
#point2 : un point de données.
def euclidean_distance(point1, point2):
    sum_squared_distance = 0
    for i in range(len(point1)):
        sum_squared_distance += math.pow(point1[i] - point2[i], 2)
    return math.sqrt(sum_squared_distance)

#Fonction
def mostSimilaPlanet(movie_query, k_recommendations):
    raw_planet_data = []
    with open('dataset.csv', 'r') as md:
        next(md)
        for line in md.readlines():
            data_row = line.strip().split(',')
            raw_planet_data.append(data_row)
    planet_recommendation_data = []
    for row in raw_planet_data:
        data_row = list(map(float, row[5:]))
        planet_recommendation_data.append(data_row)
    print(planet_recommendation_data)
    recommendation_indices, _ = knn(
        planet_recommendation_data, movie_query, k=k_recommendations,
        distance_fn=euclidean_distance, choice_fn=lambda x: None
    )
    planet_recommendations = []
    for _, index in recommendation_indices:
        planet_recommendations.append(raw_planet_data[index])
    for recommendation in planet_recommendations:
        print(recommendation[2])
mostSimilaPlanet(movie_query=query, k_recommendations=k)


#########################################################################################
#########################################################################################
#########################################################################################
#Fonction qui retroune la moyenne, le min , le max d'une des colones du fichier csv donné en paramètre
#filename : le nom du fichier csv.
#column_number : le numéro de la colonne.
def calculate_statistics(filename, column_number):
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip the header row
        values = []
        for row in reader:
            if row[column_number] != '0':
                values.append(float(row[column_number]))
        if len(values) > 0:
            mean = sum(values) / len(values)
            minimum = min(values)
            maximum = max(values)
            return mean, minimum, maximum
        else:
            return None
#########################################################################################
#########################################################################################
#########################################################################################
